In [1]:
from networkx.readwrite import json_graph  # Ferramenta para ler/escrever grafos em formato JSONfrom scipy.stats import percentileofscore   # Função para calcular o percentil de uma pontuação em uma listafrom traceback import format_exception      # Função para formatar a saída de exceçõesimport cProfile                             # Módulo para análise de desempenho de código Pythonimport json                                 # Módulo para trabalhar com dados em formato JSONimport networkx as nx                       # Biblioteca para criação, manipulação e estudo de grafos e redes complexasimport numpy as np                          # Biblioteca para operações matemáticas e manipulação de arraysimport pandas as pd                         # Biblioteca para manipulação e análise de dadosimport pstats                               # Utilitários para imprimir estatísticas de análise de desempenhoimport random                               # Módulo para geração de números aleatóriosimport sys                                  # Módulo para acessar funções e variáveis específicas do Pythonimport glob                                 # Função para encontrar todos os nomes de caminhos que correspondem a um padrão especificadoimport os                                   # Módulo para interagir com o sistema operacional

In [ ]:
# EXTRAINDO AS MATRIZES DE CONECTIVIDADE DTI DO DRIVE
files_autism=glob.glob('/content/drive/MyDrive/metricas/UCLA_Autism/*ASD*DTI*connectivity_matrix*')
files_control=glob.glob('/content/drive/MyDrive/metricas/UCLA_Autism/*TD*DTI*connectivity_matrix*')


# DROPANDO OS PRIMEIROS ELEMENTOS DA LISTA POIS SAO MATRIZES ALL
files_autism.pop(0)
files_control.pop(0)

In [2]:
# PRECISAMOS NORMALIZAR E SIMETRIZARAS MATRIZES


from sklearn.preprocessing import MinMaxScaler
mat_aut=[]
for i in files_autism:  # MATRIZES ASD (AUT)
    df = pd.read_csv(i ,sep="  ", header=None).astype(float) 
    scaler = MinMaxScaler()
    sub_norm=pd.DataFrame(scaler.fit_transform(df.T).T,columns=df.columns)
    sub_norm = sub_norm.to_numpy()
    sub = (sub_norm + sub_norm.T)/2
    #matrix=sub.to_numpy() 
    mat_aut.append(sub)

mat_control=[]
for i in files_control: # MATRIZES TD (CONTROL)
    df = pd.read_csv(i ,sep="  ", header=None).astype(float) 
    scaler = MinMaxScaler()
    sub_norm=pd.DataFrame(scaler.fit_transform(df.T).T,columns=df.columns)
    sub_norm = sub_norm.to_numpy()
    sub = (sub_norm + sub_norm.T)/2
    #matrix=sub.to_numpy() 
    mat_control.append(sub)

In [2]:
# FUNCAOUSADA PARA GERAR GRAFOS COM DENSIDADE
def G_den(matrix, d, verbose=False):
    """Returns a networkx Graph from a adjacency matrix, with a given density d
        
    Parameters
    ----------
    matrix: matrix
        A matrix of values - connectivity matrix
    
    d: float
        Density value for matrix binaziring 
    
    Returns
    -------
        NetworkX graph
        
    """
    
    #matrix i, density d. i is a matrix - ravel flatten the matrix
    np.fill_diagonal(matrix,0)
    temp = sorted(matrix.ravel(), reverse=True) # will flatten it and rank corr values
    size = len(matrix)
    cutoff = np.ceil(d * (size * (size-1))) # number of links with a given density
    tre = temp[int(cutoff)]
    G0 = nx.from_numpy_array(matrix)
    G0.remove_edges_from(list(nx.selfloop_edges(G0)))
    G1 = nx.from_numpy_array(matrix)
    for u,v,a in G0.edges(data=True):
        if (a.get('weight')) <= tre:
            G1.remove_edge(u, v)
    finaldensity = nx.density(G1)
    if verbose == True:
        print(finaldensity)
        
    return G1

In [ ]:
# AFIM DE FILTAR OS GRAFOS USANDO DISPARITY FILTER NOS VAMOS CRIAR GRAFOS CHEIOS PARA CADA GRUPO DE MATRIZES (AUT E CONTROL)

grafos_aut = []
for i in mat_aut:
  a = G_den(i, 1)
  grafos_aut.append(a)

grafos_control = []
for i in mat_control:
  a = G_den(i, 1)
  grafos_control.append(a)

In [ ]:
# AQUI VAMOS FAZER OS CORTES BASEADOS NUM VALOR DE DISPARITY FILTER 0.60, O VALOR FOI USADO AFIM DE SE OBTER UM NUMERO DE TRIANGULOS SIGNIFICANTES PARA ANALISE ESTATISTICA

def apply_disparity_filter(graph):
    """
    Aplica o filtro de disparidade ao grafo e retorna um novo grafo apenas com as arestas filtradas.
    """
    filtered_graph = nx.Graph()

    # Calcula os valores de disparidade para cada aresta do grafo
    alpha_measures = disparity_filter(graph)

    # Itera sobre as arestas do grafo e verifica o valor de disparidade
    for u, v, a in graph.edges(data=True):
        alpha = a.get('alpha', 0.0)  # Obtém o valor de disparidade da aresta, considerando 0.0 caso não exista
        if alpha in alpha_measures and alpha >= 0.60:
            filtered_graph.add_edge(u, v, alpha=alpha)

    return filtered_graph


# AQUI ESTA O LOOPING CORTANDO OS GRAFOS E SALVANDO PARA ANALISES 


# Diretório de destino para salvar os grafos filtrados
output_dir = "/content/drive/MyDrive/metricas/graficos filtrados/grafos cortados"

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Itera sobre os grafos de entrada
for i, gf in enumerate(grafos_aut):
    # Aplica o filtro de disparidade ao grafo
    filtered_graph = apply_disparity_filter(gf)

    # Define o caminho do arquivo de saída
    output_path = os.path.join(output_dir, f"grafo_cortado_{i}.txt")

    # Salva o grafo filtrado como arquivo de texto
    nx.write_adjlist(filtered_graph, output_path)

    print(f"Grafo {i} salvo em: {output_path}")



# Diretório de destino para salvar os grafos filtrados
output_dir = "/content/drive/MyDrive/metricas/graficos filtrados/grafos cortados 2"

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Itera sobre os grafos de entrada
for i, gf in enumerate(grafos_control):
    # Aplica o filtro de disparidade ao grafo
    filtered_graph = apply_disparity_filter(gf)

    # Define o caminho do arquivo de saída
    output_path = os.path.join(output_dir, f"grafo_cortado_control_{i}.txt")

    # Salva o grafo filtrado como arquivo de texto
    nx.write_adjlist(filtered_graph, output_path)

    print(f"Grafo {i} salvo em: {output_path}")


In [3]:
# AGORA VAMOS IMPORTAR OS GRAFOS CORTADOS 

In [ ]:
def extrair_grafos(diretorio):
    grafos = []
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith('.txt') and nome_arquivo.startswith('grafo_cortado_'):
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            grafo = nx.read_adjlist(caminho_arquivo)
            grafos.append(grafo)
    return grafos

# Extrair grafos da pasta de controle
diretorio_aut = '/content/drive/MyDrive/metricas/graficos filtrados/grafos cortados/'
aut_graphs = extrair_grafos(diretorio_aut)

diretorio_control = '/content/drive/MyDrive/metricas/graficos filtrados/grafos cortados 2/'
control_graphs = extrair_grafos(diretorio_control)


In [ ]:
import networkx as nx
import pandas as pd

def get_top_nodes(graphs):
  # Inputs:
# - graphs: uma lista de grafos representando redes complexas

# Outputs:
# - top_nodes_bc_df: DataFrame com os nós mais importantes e suas estatísticas de Betweenness Centrality
# - top_nodes_dc_df: DataFrame com os nós mais importantes e suas estatísticas de Degree Centrality
# - top_nodes_ec_df: DataFrame com os nós mais importantes e suas estatísticas de Eigenvector Centrality

# Funcionalidade:
# A função recebe uma lista de grafos e calcula as métricas de centralidade (Betweenness Centrality, Degree Centrality e Eigenvector Centrality) para cada grafo.
# Em seguida, identifica os 10 nós mais importantes para cada métrica em cada grafo e armazena suas informações em DataFrames separados.
# Os DataFrames finais contêm os nós mais importantes e suas estatísticas de centralidade para cada métrica.
# A função ajuda a identificar os nós mais relevantes em uma rede complexa, fornecendo insights sobre sua estrutura e importância relativa.

    # Listas para armazenar os nós mais importantes para cada métrica
    top_nodes_bc = []
    top_nodes_dc = []
    top_nodes_ec = []

    for graph in graphs:
        # Cálculo das métricas de centralidade
        bc = nx.betweenness_centrality(graph)  # Betweenness Centrality
        dc = nx.degree_centrality(graph)  # Degree Centrality
        ec = nx.eigenvector_centrality(graph)  # Eigenvector Centrality

        # Ordenação dos nós em cada métrica
        sorted_bc = sorted(bc, key=bc.get, reverse=True)
        sorted_dc = sorted(dc, key=dc.get, reverse=True)
        sorted_ec = sorted(ec, key=ec.get, reverse=True)

        # Adição dos nós mais importantes e seus valores à lista
        top_nodes_bc.extend({'nodes': node, 'BC': bc[node]} for node in sorted_bc[:10])
        top_nodes_dc.extend({'nodes': node, 'DC': dc[node]} for node in sorted_dc[:10])
        top_nodes_ec.extend({'nodes': node, 'EC': ec[node]} for node in sorted_ec[:10])

    # Criação dos DataFrames para cada métrica de centralidade
    top_nodes_bc_df = pd.DataFrame(top_nodes_bc)
    top_nodes_dc_df = pd.DataFrame(top_nodes_dc)
    top_nodes_ec_df = pd.DataFrame(top_nodes_ec)

    # Contagem e média dos valores de BC por nó
    count_bc = top_nodes_bc_df['nodes'].value_counts().reset_index()
    count_bc.columns = ['nodes', 'count']
    mean_bc = top_nodes_bc_df.groupby('nodes')['BC'].mean().reset_index()
    top_nodes_bc_df = pd.merge(count_bc, mean_bc, on='nodes')

    # Contagem e média dos valores de DC por nó
    count_dc = top_nodes_dc_df['nodes'].value_counts().reset_index()
    count_dc.columns = ['nodes', 'count']
    mean_dc = top_nodes_dc_df.groupby('nodes')['DC'].mean().reset_index()
    top_nodes_dc_df = pd.merge(count_dc, mean_dc, on='nodes')

    # Contagem e média dos valores de EC por nó
    count_ec = top_nodes_ec_df['nodes'].value_counts().reset_index()
    count_ec.columns = ['nodes', 'count']
    mean_ec = top_nodes_ec_df.groupby('nodes')['EC'].mean().reset_index()
    top_nodes_ec_df = pd.merge(count_ec, mean_ec, on='nodes')

    return top_nodes_bc_df, top_nodes_dc_df, top_nodes_ec_df


In [ ]:
# TOMANDO OS HUBS DOS GRAFOS CORTADOS
hubs_g_aut = get_top_nodes(aut_graphs)
hubs_g_control = get_top_nodes(control_graphs)

#HUBS PARA O GRUPO CONTROL
hubs_g_control_bc = hubs_g_control[0]
hubs_g_control_dc = hubs_g_control[1]
hubs_g_control_ec = hubs_g_control[2]

#HUBS PARA O GRUPO AUT
hubs_g_aut_bc = hubs_g_aut[0]
hubs_g_aut_dc = hubs_g_aut[1]
hubs_g_aut_ec = hubs_g_aut[2]

In [4]:
# CRIANDO OS HYPERGRAFOS PARA CALCULAR OS HUBS

# O RETURN DA FUNCAO SAO AS LISTAS DE CLIQUES POIS ESTAMOS INTERESSADO NOS HUBS

def HG_aut(graphs):
    HG_aut = []
    tri_aut = []
    for i, G in enumerate(graphs):
        all_cliques = nx.enumerate_all_cliques(G)
        triangles = [x for x in all_cliques if len(x) == 3]
        tri_aut.append(triangles)
        size = len(triangles)
        Hypergraph = np.empty((size, size))
        for i in range(0, len(triangles)):
            for j in range(0, len(triangles)):
                a = set(triangles[i])
                b = set(triangles[j])
                if len(a & b) == 2:
                    Hypergraph[i, j] = int(1)
        HG_aut.append(Hypergraph.astype(int))
        # Save each triangle set to a separate file
        for j, triangle_set in enumerate(triangles):
            triangle_set = np.array(triangle_set, dtype=int)

    return tri_aut

def HG_control(graphs):
    HG_control = []
    tri_control = []
    for i, G in enumerate(graphs):
        all_cliques = nx.enumerate_all_cliques(G)
        triangles = [x for x in all_cliques if len(x) == 3]
        tri_control.append(triangles)
        size = len(triangles)
        Hypergraph = np.empty((size, size))
        for i in range(0, len(triangles)):
            for j in range(0, len(triangles)):
                a = set(triangles[i])
                b = set(triangles[j])
                if len(a & b) == 2:
                    Hypergraph[i, j] = int(1)
        HG_control.append(Hypergraph.astype(int))
        # Save each triangle set to a separate file
        for j, triangle_set in enumerate(triangles):
            triangle_set = np.array(triangle_set, dtype=int)

    return tri_control

In [ ]:
# CLIQUES PARA OS GRUPOS CONTROL E AUT


tri_control = HG_control(control_graphs) 
tri_aut = HG_aut(aut_graphs)

In [ ]:
# CALCULANDO OS HUBS DOS GRAFOS E HYPERGRAFOS

hubs_aut_hp = []
for cliques in tri_aut:
    c = calculate_dual_centralities(cliques)
    hubs_aut_hp.append(c)

hubs_control_hp = []
for cliques in tri_control:
    c = calculate_dual_centralities(cliques)
    hubs_control_hp.append(c)

In [ ]:
# SEPARANDO OS DATAFRAMES POR METRICAS

def split_dataframe(df, metric):
    # Resetar o índice para tornar a coluna 'Clique' em uma coluna regular
    df = df.reset_index()

    # Ordenar o DataFrame pela métrica desejada em ordem decrescente
    sorted_df = df.sort_values(by=metric, ascending=False)

    # Criar o DataFrame separado contendo as colunas 'Clique' e a métrica
    result_df = pd.DataFrame({'Clique': sorted_df['Clique'], metric: sorted_df[metric]})

    return result_df


hubs_aut_hp_split = []
for df in hubs_aut_hp:
    df_bc_aut = split_dataframe(df, 'BC')
    df_dc_aut = split_dataframe(df, 'DC')
    df_ec_aut = split_dataframe(df, 'EC')

    # Adicionar os DataFrames separados à lista hubs_aut_hp_split
    hubs_aut_hp_split.append(df_bc_aut)
    hubs_aut_hp_split.append(df_dc_aut)
    hubs_aut_hp_split.append(df_ec_aut)


hubs_control_hp_split = []
for df in hubs_control_hp:
    df_bc_control = split_dataframe(df, 'BC')
    df_dc_control = split_dataframe(df, 'DC')
    df_ec_control = split_dataframe(df, 'EC')

    # Adicionar os DataFrames separados à lista hubs_aut_hp_split
    hubs_control_hp_split.append(df_bc_control)
    hubs_control_hp_split.append(df_dc_control)
    hubs_control_hp_split.append(df_ec_control)

In [ ]:
# ASSOCIANDO OS CLIQUES AS REGIOES DA LISTA NOMES

def associar_cliques_regioes(df, lista_nomes, metrica):
    cliques_regioes = []
    for clique in df['Clique']:
        regioes_clique = [lista_nomes[int(indice)-1] for indice in clique]
        cliques_regioes.append(regioes_clique)

    df_resultado = pd.DataFrame({'Clique': df['Clique'], metrica: df[metrica], 'Regioes': cliques_regioes})
    return df_resultado


# HUBS COM NOMES DAS REGIOES DOS HYPERGRAFOS

df_bc_aut_nomes = associar_cliques_regioes(df_bc_aut, nomes, 'BC')
df_dc_aut_nomes = associar_cliques_regioes(df_dc_aut, nomes, 'DC')
df_ec_aut_nomes = associar_cliques_regioes(df_ec_aut, nomes, 'EC')

df_bc_control_nomes = associar_cliques_regioes(df_bc_control, nomes, 'BC')
df_dc_control_nomes = associar_cliques_regioes(df_dc_control, nomes, 'DC')
df_ec_control_nomes = associar_cliques_regioes(df_ec_control, nomes, 'EC')

In [ ]:
###### FAZENDO A MESMA COISA AGORA PARA OS GRAFOS E HYPERGRAFOS COM DENSIDADE 5% ########
### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ ##############
# A PRIMEIRA ETAPA DE EXTRAIR AS MATRIZES E SIMETRIZAR É A MESMA PARA AMBOS OS METODOS

In [ ]:
# USAMOS A MESMA FUNCAO PARA CRIAR OS GRAFOS COM DENSIDADE 5% PARA AMBOS OS GRUPOS

grafos_aut = []
for i in mat_aut:
  a = G_den(i, 0.05)
  grafos_aut.append(a)

grafos_controll = []
for i in mat_control:
  a = G_den(i, 0.05)
  grafos_controll.append(a)

In [ ]:
# FUNCO QUE CALCULA AS CENTRALIDADES É A MESMA

def get_top_nodes(graphs):
    top_nodes_bc = []
    top_nodes_dc = []
    top_nodes_ec = []

    for graph in graphs:
        # Cálculo das métricas de centralidade
        bc = nx.betweenness_centrality(graph)
        dc = nx.degree_centrality(graph)
        ec = nx.eigenvector_centrality(graph)

        # Ordenação dos nós em cada métrica
        sorted_bc = sorted(bc, key=bc.get, reverse=True)
        sorted_dc = sorted(dc, key=dc.get, reverse=True)
        sorted_ec = sorted(ec, key=ec.get, reverse=True)
  
        # Adição dos nós mais importantes e seus valores à lista
        top_nodes_bc.extend({'nodes': node, 'BC': bc[node]} for node in sorted_bc[:10])
        top_nodes_dc.extend({'nodes': node, 'DC': dc[node]} for node in sorted_dc[:10])
        top_nodes_ec.extend({'nodes': node, 'EC': ec[node]} for node in sorted_ec[:10])

    top_nodes_bc_df = pd.DataFrame(top_nodes_bc)
    top_nodes_dc_df = pd.DataFrame(top_nodes_dc)
    top_nodes_ec_df = pd.DataFrame(top_nodes_ec)

    # Contagem e média dos valores de BC por nó
    count_bc = top_nodes_bc_df['nodes'].value_counts().reset_index()
    count_bc.columns = ['nodes', 'count']
    mean_bc = top_nodes_bc_df.groupby('nodes')['BC'].mean().reset_index()
    top_nodes_bc_df = pd.merge(count_bc, mean_bc, on='nodes')

    count_dc = top_nodes_dc_df['nodes'].value_counts().reset_index()
    count_dc.columns = ['nodes', 'count']
    mean_dc = top_nodes_dc_df.groupby('nodes')['DC'].mean().reset_index()
    top_nodes_dc_df = pd.merge(count_dc, mean_dc, on='nodes')

    count_ec = top_nodes_ec_df['nodes'].value_counts().reset_index()
    count_ec.columns = ['nodes', 'count']
    mean_ec = top_nodes_ec_df.groupby('nodes')['EC'].mean().reset_index()
    top_nodes_ec_df = pd.merge(count_ec, mean_ec, on='nodes')

    return top_nodes_bc_df, top_nodes_dc_df, top_nodes_ec_df


In [ ]:
# CALCULANO OS HUBS PARA OS GRAFOS 5%

hubs_g_aut = get_top_nodes(grafos_aut)
hubs_g_control = get_top_nodes(grafos_controll)


In [ ]:
# SEPARANDO OS HUBS POR METRICAS

hubs_g_control_bc = hubs_g_control[0].head(40)
hubs_g_control_dc = hubs_g_control[1].head(40)
hubs_g_control_ec = hubs_g_control[2].head(40)

hubs_g_aut_bc = hubs_g_aut[0].head(40)
hubs_g_aut_dc = hubs_g_aut[1].head(40)
hubs_g_aut_ec = hubs_g_aut[2].head(40)

In [ ]:
# ASSOCIANDO OS HUBS COM OS NOMES DAS REGIOES

def associate_names(df, nomes):
    node_names = []
    for node in df['nodes']:
        node = int(node)
        if node <= len(nomes):
            node_names.append(nomes[node - 1])
        else:
            node_names.append(None)
    df['regiao'] = node_names
    return df

hubs_c_bc_aut_nomes = associate_names(hubs_g_aut_bc, nomes)
hubs_c_dc_aut_nomes = associate_names(hubs_g_aut_dc, nomes)
hubs_c_ec_aut_nomes = associate_names(hubs_g_aut_ec, nomes)

hubs_c_bc_control_nomes = associate_names(hubs_g_control_bc, nomes)
hubs_c_dc_control_nomes = associate_names(hubs_g_control_dc, nomes)
hubs_c_ec_control_nomes = associate_names(hubs_g_control_ec, nomes)

In [ ]:
# PARA GERAR OS HYPERGRAFOS USAMOS A MESMA FUNCAO ANTERIOR

In [ ]:
tri_control = HG_control(grafos_controll)
tri_aut = HG_aut(grafos_aut)



In [ ]:
## ESSAS TRES FUNCOES ACIMA FUNCIONAM JUNTAS  

def construct_dual_graph(cliques): # AQUI CONSTRUIMOS OS GRAFOS DUAIS PARA CADA HYPERGRAFO A PARTIR DOS CLIQUES 
    dual_graph = nx.Graph()
    node_hyperedge_map = {}
    for hyperedge, clique in enumerate(cliques):
        for node in clique:
            if node not in node_hyperedge_map:
                node_hyperedge_map[node] = set()
            node_hyperedge_map[node].add(hyperedge)

    for hyperedges in node_hyperedge_map.values():
        for hyperedge1 in hyperedges:
            for hyperedge2 in hyperedges:
                if hyperedge1 != hyperedge2:
                    dual_graph.add_edge(hyperedge1, hyperedge2)

    return dual_graph


def calculate_centralities(graph): # AQUI CALCULAMOS AS CENTRALIDADES DE CADA GRAFO DUAL
    bc = nx.betweenness_centrality(graph)
    dc = nx.degree_centrality(graph)
    ec = nx.eigenvector_centrality(graph, max_iter=1000)

    df = pd.DataFrame({'BC': bc, 'DC': dc, 'EC': ec})
    df.index.name = 'Node'

    return df


def calculate_dual_centralities(cliques): # AQUI CALCULAMOS OS HUBS DOS CLIQUES
    dual_graph = construct_dual_graph(cliques)

    centralities_df = calculate_centralities(dual_graph)

    result_df = pd.DataFrame({'Clique': cliques})
    result_df = result_df.join(centralities_df)

    return result_df




In [ ]:
# CALCULANDO OS HUBS DOS HYPERGRAFOS


hubs_aut_hp = []
for cliques in tri_aut:
    c = calculate_dual_centralities(cliques)
    hubs_aut_hp.append(c)

hubs_control_hp = []
for cliques in tri_control:
    c = calculate_dual_centralities(cliques)
    hubs_control_hp.append(c)


In [ ]:
#SEPARANDO OS HUBS POR METRICAS

def split_dataframe(df, metric):
    # Resetar o índice para tornar a coluna 'Clique' em uma coluna regular
    df = df.reset_index()

    # Ordenar o DataFrame pela métrica desejada em ordem decrescente
    sorted_df = df.sort_values(by=metric, ascending=False)

    # Criar o DataFrame separado contendo as colunas 'Clique' e a métrica
    result_df = pd.DataFrame({'Clique': sorted_df['Clique'], metric: sorted_df[metric]})

    return result_df


hubs_aut_hp_split = []
for df in hubs_aut_hp:
    df_bc_aut = split_dataframe(df, 'BC')
    df_dc_aut = split_dataframe(df, 'DC')
    df_ec_aut = split_dataframe(df, 'EC')

    # Adicionar os DataFrames separados à lista hubs_aut_hp_split
    hubs_aut_hp_split.append(df_bc_aut)
    hubs_aut_hp_split.append(df_dc_aut)
    hubs_aut_hp_split.append(df_ec_aut)


hubs_control_hp_split = []
for df in hubs_control_hp:
    df_bc_control = split_dataframe(df, 'BC')
    df_dc_control = split_dataframe(df, 'DC')
    df_ec_control = split_dataframe(df, 'EC')

    # Adicionar os DataFrames separados à lista hubs_aut_hp_split
    hubs_control_hp_split.append(df_bc_control)
    hubs_control_hp_split.append(df_dc_control)
    hubs_control_hp_split.append(df_ec_control)

In [ ]:
#ASSOCIANDO OS CLIQUES A REGIOES


def associar_cliques_regioes(df, lista_nomes, metrica):
    cliques_regioes = []
    for clique in df['Clique']:
        regioes_clique = [lista_nomes[indice-1] for indice in clique]
        cliques_regioes.append(regioes_clique)

    df_resultado = pd.DataFrame({'Clique': df['Clique'], metrica: df[metrica], 'Regioes': cliques_regioes})
    return df_resultado

df_bc_aut_nomes = associar_cliques_regioes(df_bc_aut, nomes, 'BC')
df_dc_aut_nomes = associar_cliques_regioes(df_dc_aut, nomes, 'DC')
df_ec_aut_nomes = associar_cliques_regioes(df_ec_aut, nomes, 'EC')

df_bc_control_nomes = associar_cliques_regioes(df_bc_control, nomes, 'BC')
df_dc_control_nomes = associar_cliques_regioes(df_dc_control, nomes, 'DC')
df_ec_control_nomes = associar_cliques_regioes(df_ec_control, nomes, 'EC')

In [6]:
####################################################################
#### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$######
# AQUI VAMOS SEPARAR OS HUBS POR DATAFRAMES

In [ ]:
# COLOCANDO _ ENTRE OS NOMES DAS REGIOES PARA EVITAR OVERCLICK DAS PALAVRAS RIGHT, LEFT, ...

def adicionar_underscore(nomes):
    nomes_com_underscore = []
    
    for nome in nomes:
        nome_com_underscore = nome.replace(" ", "_")
        nomes_com_underscore.append(nome_com_underscore)
    
    return nomes_com_underscore

nomes = adicionar_underscore(nomes)

# AQUI IMPORTEI OS HUBS QUE FORAM COMPUTADOS ANTERIORMENTE

base_path = '/content/drive/MyDrive/metricas/graficos filtrados/dfs'
folders = ['hubs g disp', 'hubs hp disp', 'hubs g 5%', 'hubs hp 5%']
hubs_g_disp_list = []
hubs_hp_disp_list = []
hubs_g_5pct_list = []
hubs_hp_5pct_list = []

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    files = os.listdir(folder_path)

    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)

            if folder == 'hubs g disp':
                hubs_g_disp_list.append(df)
            elif folder == 'hubs hp disp':
              
              hubs_hp_disp_list.append(df)
            elif folder == 'hubs g 5%':
                hubs_g_5pct_list.append(df)
            elif folder == 'hubs hp 5%':
                hubs_hp_5pct_list.append(df)


In [ ]:
# HUBS DOS GRAFOS DISPARITY FILTER

hubs_g_disp_aut_bc = hubs_g_disp_list[3]
hubs_g_disp_aut_dc = hubs_g_disp_list[4]
hubs_g_disp_aut_ec = hubs_g_disp_list[5]
hubs_g_disp_control_bc = hubs_g_disp_list[0]
hubs_g_disp_control_dc = hubs_g_disp_list[1]
hubs_g_disp_control_ec = hubs_g_disp_list[2]

In [ ]:
# HUBS DOS HYPERGRAFOS DISPARITY FILTER

hubs_hp_disp_aut_bc = hubs_hp_disp_list[0]
hubs_hp_disp_aut_dc = hubs_hp_disp_list[1]
hubs_hp_disp_aut_ec = hubs_hp_disp_list[2]

hubs_hp_disp_control_bc = hubs_hp_disp_list[3]
hubs_hp_disp_control_dc = hubs_hp_disp_list[4]
hubs_hp_disp_control_ec = hubs_hp_disp_list[4]

In [ ]:
# HUBS DOS GRAFOS 5% 

hubs_g_5pct_aut_bc = hubs_g_5pct_list[0]
hubs_g_5pct_aut_dc = hubs_g_5pct_list[1]
hubs_g_5pct_aut_ec = hubs_g_5pct_list[2]

hubs_g_5pct_control_bc = hubs_g_5pct_list[3]
hubs_g_5pct_control_dc = hubs_g_5pct_list[4]
hubs_g_5pct_control_ec = hubs_g_5pct_list[4]

In [ ]:
# HUBS DOS HYPERGRAFOS 5%

hubs_hp_5pct_bc_aut = hubs_hp_5pct_list[0]
hubs_hp_5pct_dc_aut = hubs_hp_5pct_list[1]
hubs_hp_5pct_ec_aut = hubs_hp_5pct_list[2]

hubs_hp_5pct_bc_control = hubs_hp_5pct_list[3]
hubs_hp_5pct_dc_control = hubs_hp_5pct_list[4]
hubs_hp_5pct_ec_control = hubs_hp_5pct_list[5]

In [ ]:
import pandas as pd
import ast

def encontrar_nos_comuns_5(hubs_df, clique_df): # AQUI CALCULAMOS OS NODES COMUNS NOS HUBS CLIQUES QUE TAMBEM APARECEM NOS HUBS DOS GRAFOS 
    # Extrair os nós dos dataframes
    nodes_hubs = hubs_df['nodes'].tolist()
    nodes_clique = [ast.literal_eval(clique) for clique in clique_df['Clique']]

    # Flattening para obter uma lista plana de nós
    flattened_nodes_clique = [node for sublist in nodes_clique for node in sublist]

    # Encontrar nós comuns
    nos_comuns = list(set(nodes_hubs) & set(flattened_nodes_clique))

    # Criar um dicionário para armazenar as informações dos nós comuns
    nos_dict = {'nodes': nos_comuns, 'freq': [], 'regiao': []}

    # Preencher o dicionário com a frequência e região associada a cada nó
    for node in nos_comuns:
        freq = flattened_nodes_clique.count(node)
        regiao = hubs_df.loc[hubs_df['nodes'] == node, 'regiao'].values[0]
        nos_dict['freq'].append(freq)
        nos_dict['regiao'].append(regiao)

    # Preencher com nós que não aparecem em comum com frequência 0
    for node in range(1, 265):
        if node not in nos_dict['nodes']:
            nos_dict['nodes'].append(node)
            nos_dict['freq'].append(0)
            nos_dict['regiao'].append('')

    # Criar o dataframe com os nós comuns, frequência e região
    df = pd.DataFrame(nos_dict)

    # Ordenar o dataframe em ordem decrescente de frequência
    df = df.sort_values('freq', ascending=False)

    return df


# NODES DOS HUBS DOS HYPERGRAFOS QUE SAO HUBS DOS GRAFOS TAMBEM PARA 5%

freq_hubs_aut_bc_5 = encontrar_nos_comuns_5(hubs_g_5pct_aut_bc, hubs_hp_5pct_bc_aut).head(15)
freq_hubs_aut_dc_5 = encontrar_nos_comuns_5(hubs_g_5pct_aut_dc, hubs_hp_5pct_dc_aut).head(15)
freq_hubs_aut_ec_5 = encontrar_nos_comuns_5(hubs_g_5pct_aut_ec, hubs_hp_5pct_ec_aut).head(15)

freq_hubs_control_bc_5 = encontrar_nos_comuns_5(hubs_g_5pct_control_bc, hubs_hp_5pct_bc_control).head(15)
freq_hubs_control_dc_5 = encontrar_nos_comuns_5(hubs_g_5pct_control_dc, hubs_hp_5pct_dc_control).head(15)
freq_hubs_control_ec_5 = encontrar_nos_comuns_5(hubs_g_5pct_control_ec, hubs_hp_5pct_ec_control).head(15)

In [ ]:
import pandas as pd

def encontrar_nos_comuns(hubs_df, clique_df): # FAZEMOS O MESMO AQUI, PRECISOU ADAPTAR A FUNCAO POIS OS CLIQUES ESTAO ENTRE ' '
    # Extrair os nós dos dataframes
    nodes_hubs = hubs_df['nodes'].tolist()
    nodes_clique = [clique.strip("[]'").split("', '") for clique in clique_df['Clique']]

    # Criar um dicionário para armazenar as informações dos nós comuns
    nos_dict = {'nodes': [], 'freq': [], 'regiao': []}

    # Percorrer os nós do dataframe hubs_df
    for node in nodes_hubs:
        freq = sum([str(node) in clique for clique in nodes_clique])
        regiao = hubs_df.loc[hubs_df['nodes'] == node, 'regiao'].values[0]
        nos_dict['nodes'].append(node)
        nos_dict['freq'].append(freq)
        nos_dict['regiao'].append(regiao)

    # Preencher com nós que não aparecem em comum com frequência 0
    for node in range(1, 265):
        if node not in nos_dict['nodes']:
            nos_dict['nodes'].append(node)
            nos_dict['freq'].append(0)
            nos_dict['regiao'].append('')

    # Criar o dataframe com os nós comuns, frequência e região
    df = pd.DataFrame(nos_dict)

    # Ordenar o dataframe em ordem decrescente de frequência
    df = df.sort_values('freq', ascending=False)

    return df
# NODES DOS HUBS DOS HYPERGRAFOS QUE SAO HUBS DOS GRAFOS TAMBEM PARA DISPARITY

freq_hubs_aut_disp_bc = encontrar_nos_comuns(hubs_g_disp_aut_bc, hubs_hp_disp_aut_bc).head(15)
freq_hubs_aut_disp_dc = encontrar_nos_comuns(hubs_g_disp_aut_dc, hubs_hp_disp_aut_dc).head(15)
freq_hubs_aut_disp_ec = encontrar_nos_comuns(hubs_g_disp_aut_ec, hubs_hp_disp_aut_ec).head(15)

freq_hubs_control_disp_bc = encontrar_nos_comuns(hubs_g_disp_control_bc, hubs_hp_disp_control_bc).head(15)
freq_hubs_control_disp_dc = encontrar_nos_comuns(hubs_g_disp_control_dc, hubs_hp_disp_control_dc).head(15)
freq_hubs_control_disp_ec = encontrar_nos_comuns(hubs_g_disp_control_ec, hubs_hp_disp_control_ec).head(15)

In [ ]:
# AGORA VOU IMPRIMIR OS NODES COMUNS


# Métrica BC
cliques_bc = hubs_hp_disp_aut_bc['Clique'].apply(eval)  # Converter as strings para listas
cliques_bc = cliques_bc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_bc = set(cliques_bc.sum()) & set(hubs_g_disp_aut_bc['nodes'].astype(str))
common_nodes_bc_aut_d = set(nomes[int(node)-1] for node in common_nodes_bc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (BC):", common_nodes_bc_aut_d)

# Métrica DC
cliques_dc = hubs_hp_disp_aut_dc['Clique'].apply(eval)  # Converter as strings para listas
cliques_dc = cliques_dc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_dc = set(cliques_dc.sum()) & set(hubs_g_disp_aut_dc['nodes'].astype(str))
common_nodes_dc_aut_d = set(nomes[int(node)-1] for node in common_nodes_dc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (DC):", common_nodes_dc_aut_d)

# Métrica EC
cliques_ec = hubs_hp_disp_aut_ec['Clique'].apply(eval)  # Converter as strings para listas
cliques_ec = cliques_ec.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_ec = set(cliques_ec.sum()) & set(hubs_g_disp_aut_ec['nodes'].astype(str))
common_nodes_ec_aut_d = set(nomes[int(node)-1] for node in common_nodes_ec if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (EC):", common_nodes_ec_aut_d)


In [ ]:
# Métrica BC
cliques_bc = hubs_hp_disp_control_bc['Clique'].apply(eval)  # Converter as strings para listas
cliques_bc = cliques_bc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_bc = set(cliques_bc.sum()) & set(hubs_g_disp_control_bc['nodes'].astype(str))
common_nodes_bc_control_d = set(nomes[int(node)-1] for node in common_nodes_bc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (BC):", common_nodes_bc_control_d)

# Métrica DC
cliques_dc = hubs_hp_disp_control_dc['Clique'].apply(eval)  # Converter as strings para listas
cliques_dc = cliques_dc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_dc = set(cliques_dc.sum()) & set(hubs_g_disp_control_dc['nodes'].astype(str))
common_nodes_dc_control_d = set(nomes[int(node)-1] for node in common_nodes_dc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (DC):", common_nodes_dc_control_d)

# Métrica EC
cliques_ec = hubs_hp_disp_control_ec['Clique'].apply(eval)  # Converter as strings para listas
cliques_ec = cliques_ec.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_ec = set(cliques_ec.sum()) & set(hubs_g_disp_control_ec['nodes'].astype(str))
common_nodes_ec_control_d = set(nomes[int(node)-1] for node in common_nodes_ec if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (EC):", common_nodes_ec_control_d)

In [ ]:


# Nós exclusivos dos hypergrafos (EC)
exclusive_nodes_hypergraph_ec = set(hubs_hp_disp_aut_ec['Clique'].sum()) - set(hubs_g_disp_aut_ec['nodes'].astype(str))
exclusive_nodes_hypergraph_ec_aut_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_ec if node.isdigit())
print("Nós exclusivos dos hypergrafos (EC):", exclusive_nodes_hypergraph_ec_aut_d)

# Nós exclusivos dos hypergrafos (BC)
exclusive_nodes_hypergraph_bc = set(hubs_hp_disp_aut_bc['Clique'].sum()) - set(hubs_g_disp_aut_bc['nodes'].astype(str))
exclusive_nodes_hypergraph_bc_aut_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_bc if node.isdigit())
print("Nós exclusivos dos hypergrafos (BC):", exclusive_nodes_hypergraph_bc_aut_d)

# Nós exclusivos dos hypergrafos (DC)
exclusive_nodes_hypergraph_dc = set(hubs_hp_disp_aut_dc['Clique'].sum()) - set(hubs_g_disp_aut_dc['nodes'].astype(str))
exclusive_nodes_hypergraph_dc_aut_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_dc if node.isdigit())
print("Nós exclusivos dos hypergrafos (DC):", exclusive_nodes_hypergraph_dc_aut_d)


In [ ]:


# Nós exclusivos dos hypergrafos (EC)
exclusive_nodes_hypergraph_ec = set(hubs_hp_disp_control_ec['Clique'].sum()) - set(hubs_g_disp_control_ec['nodes'].astype(str))
exclusive_nodes_hypergraph_ec_control_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_ec if node.isdigit())
print("Nós exclusivos dos hypergrafos (EC):", exclusive_nodes_hypergraph_ec_control_d)

# Nós exclusivos dos hypergrafos (BC)
exclusive_nodes_hypergraph_bc = set(hubs_hp_disp_control_bc['Clique'].sum()) - set(hubs_g_disp_control_bc['nodes'].astype(str))
exclusive_nodes_hypergraph_bc_control_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_bc if node.isdigit())
print("Nós exclusivos dos hypergrafos (BC):", exclusive_nodes_hypergraph_bc_control_d)

# Nós exclusivos dos hypergrafos (DC)
exclusive_nodes_hypergraph_dc = set(hubs_hp_disp_control_dc['Clique'].sum()) - set(hubs_g_disp_control_dc['nodes'].astype(str))
exclusive_nodes_hypergraph_dc_control_d = set(nomes[int(node)] for node in exclusive_nodes_hypergraph_dc if node.isdigit())
print("Nós exclusivos dos hypergrafos (DC):", exclusive_nodes_hypergraph_dc_control_d)


In [ ]:

# Nós exclusivos dos hypergrafos (EC)
exclusive_nodes_hypergraph_ec = set(hubs_hp_5pct_ec_aut['Clique'].sum()) - set(hubs_g_5pct_aut_ec['nodes'].astype(str))
exclusive_nodes_hypergraph_ec_aut_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_ec if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (EC):", exclusive_nodes_hypergraph_ec_aut_5)

# Nós exclusivos dos hypergrafos (BC)
exclusive_nodes_hypergraph_bc = set(hubs_hp_5pct_bc_aut['Clique'].sum()) - set(hubs_g_5pct_aut_bc['nodes'].astype(str))
exclusive_nodes_hypergraph_bc_aut_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_bc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (BC):", exclusive_nodes_hypergraph_bc_aut_5)

# Nós exclusivos dos hypergrafos (DC)
exclusive_nodes_hypergraph_dc = set(hubs_hp_5pct_dc_aut['Clique'].sum()) - set(hubs_g_5pct_aut_dc['nodes'].astype(str))
exclusive_nodes_hypergraph_dc_aut_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_dc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (DC):", exclusive_nodes_hypergraph_dc_aut_5)


In [ ]:
# Nós exclusivos dos hypergrafos (EC)
exclusive_nodes_hypergraph_ec = set(hubs_hp_5pct_ec_control['Clique'].sum()) - set(hubs_g_5pct_control_ec['nodes'].astype(str))
exclusive_nodes_hypergraph_ec_control_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_ec if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (EC):", exclusive_nodes_hypergraph_ec_control_5)

# Nós exclusivos dos hypergrafos (BC)
exclusive_nodes_hypergraph_bc = set(hubs_hp_5pct_bc_control['Clique'].sum()) - set(hubs_g_5pct_control_bc['nodes'].astype(str))
exclusive_nodes_hypergraph_bc_control_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_bc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (BC):", exclusive_nodes_hypergraph_bc_control_5)

# Nós exclusivos dos hypergrafos (DC)
exclusive_nodes_hypergraph_dc = set(hubs_hp_5pct_dc_control['Clique'].sum()) - set(hubs_g_5pct_control_dc['nodes'].astype(str))
exclusive_nodes_hypergraph_dc_control_5 = set(nomes[int(node)-1] for node in exclusive_nodes_hypergraph_dc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós exclusivos dos hypergrafos (DC):", exclusive_nodes_hypergraph_dc_control_5)


In [ ]:


# Métrica BC
cliques_bc = hubs_hp_5pct_bc_aut['Clique'].apply(eval)  # Converter as strings para listas
cliques_bc = cliques_bc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_bc = set(cliques_bc.sum()) & set(hubs_g_5pct_aut_bc['nodes'].astype(str))
common_nodes_bc_aut_5  = set(nomes[int(node)-1] for node in common_nodes_bc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (BC):", common_nodes_bc_aut_5 )

# Métrica DC
cliques_dc = hubs_hp_5pct_dc_aut['Clique'].apply(eval)  # Converter as strings para listas
cliques_dc = cliques_dc.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_dc = set(cliques_dc.sum()) & set(hubs_g_5pct_aut_dc['nodes'].astype(str))
common_nodes_dc_aut_5  = set(nomes[int(node)-1] for node in common_nodes_dc if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (DC):", common_nodes_dc_aut_5)

# Métrica EC
cliques_ec = hubs_hp_5pct_ec_aut['Clique'].apply(eval)  # Converter as strings para listas
cliques_ec = cliques_ec.apply(lambda x: [str(node) for node in x])  # Converter os nós para strings
common_nodes_ec = set(cliques_ec.sum()) & set(hubs_g_5pct_aut_ec['nodes'].astype(str))
common_nodes_ec_aut_5  = set(nomes[int(node)-1] for node in common_nodes_ec if node.isdigit() and int(node)-1 < len(nomes))
print("Nós comuns (EC):", common_nodes_ec_aut_5)

In [ ]:
# AQUI GEREI OS WORDCLOUDS

from wordcloud import WordCloud

def plot_wordclouds(common_nodes_bc, common_nodes_dc, common_nodes_ec):
    # Criar WordCloud para a Métrica BC
    wordcloud_bc = WordCloud(background_color='white', width=400, height=300).generate(' '.join(common_nodes_bc))

    # Criar WordCloud para a Métrica DC
    wordcloud_dc = WordCloud(background_color='white', width=400, height=300).generate(' '.join(common_nodes_dc))

    # Criar WordCloud para a Métrica EC
    wordcloud_ec = WordCloud(background_color='white', width=400, height=300).generate(' '.join(common_nodes_ec))

    # Configurar a figura com três subplots lado a lado
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    # Plotar WordCloud para a Métrica BC
    axes[0].imshow(wordcloud_bc, interpolation='bilinear')
    axes[0].set_title('Hubs comuns G x HG TD BC')
    axes[0].axis('off')

    # Plotar WordCloud para a Métrica DC
    axes[1].imshow(wordcloud_dc, interpolation='bilinear')
    axes[1].set_title('Hubs comuns G x HG TD DC')
    axes[1].axis('off')

    # Plotar WordCloud para a Métrica EC
    axes[2].imshow(wordcloud_ec, interpolation='bilinear')
    axes[2].set_title('Hubs comuns G x HG TD EC')
    axes[2].axis('off')

    # Ajustar espaçamento entre os subplots
    plt.tight_layout()

    # Exibir a figura
    plt.show()